In [ ]:
import os
import pandas as pd
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

<h3>Variables</h3>

In [ ]:
RUTA_XLSX_PREMARCA = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoReconocimiento\3.Geoenriquecimiento_PreMarcas\zInsumos\Premarcas_Normalizadas_MariaLaBaja.xlsx"
RUTA_FILEGDB = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb"
RUTA_XLSX = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoReconocimiento\3.Geoenriquecimiento_PreMarcas\Reporte_PreMarcas"

NOMBRE_XLSX = 'PreMarcas_MariaLaBaja.xlsx'
NOMBRE_INFORMALIDADES_XLSX = 'PreMarcas_Informalidades.xlsx'

TIPO_COLUMNA = {'condicion_predio': str,
    'predio': str
    }

INCONSISTENCIAS_GENERALES = ['npn',
    'predio',
    'condicion_predio',
    'condicion_de_propiedad',
    'destino',
    'total_errores_de_propietario',
    'total_errores_de_documento',
    'total_errores_direccion',
    'total_errores_construcciones',
    'total_errores_destino',
    'total_inconsistencias_generales',
    # 'total_inconsistencias_alfanumericas',
    'inconsistencias_juridicas'
]

INCONSISTENCIAS_GEOGRAFICAS = ['npn',
    'predio',
    'condicion_predio',
    'condicion_de_propiedad',
    'destino',
    'area_grafica_(m2)',
    'diferencia_(m2)',
    'diferencia_porcentaje',
    'error_diferencia_1_porciento',
]

INCONSISTENCIAS_SALIDA_XLSX = ['npn',
    'zona',
    'sector',
    'manzana_vereda',
    'predio',
    'condicion_predio',
    'condicion_de_propiedad',
    'destino_economico',
    'destino',
    'direccion',
    'folio',
    'area_terreno',
    'area_construccion',
    'cantidad_propietarios_sin_identificacion',
    'cantidad_error_codificacion_propiertario',
    'cantidad_mismo_propietario_diferente_cedula',
    'total_errores_de_propietario',
    'cantidad_misma_cedula_diferente_propietario',
    'cantidad_error_tipo_documento',
    'cantidad_codificacion_documento',
    'total_errores_de_documento',
    'predios_sin_direccion',
    'error_en_codificacion_de_direccion',
    'total_errores_direccion',
    'construcciones_sin_puntaje',
    'error_puntaje_construccion_anexa',
    'error_codificacion_usos_construcciones_ph',
    'ph_no_construido',
    'total_errores_construcciones',
    'predios_sin_destino',
    'predios_con_destino_lote_y_area_construida',
    'destinos_no_lote_sin_area_construida',
    'total_errores_destino',
    'mejoras_sin_matriz',
    'registros_repetidos',
    'predios_sin_area_de_terreno',
    'mejoras_sin_area_construida',
    'mejoras_con_area_de_terreno',
    'total_inconsistencias_generales',
    'total_inconsistencias_alfanumericas',
    'predios_sin_folio',
    'folio_antiguo',
    'formato',
    'circulo_registral',
    'predios_con_folio_duplicado',
    'mejoras_con_folio',
    'inconsistencias_juridicas',
    'omision',
    'error_de_omision',
    'area_grafica_(m2)',
    'diferencia_(m2)',
    'diferencia_porcentaje',
    'error_diferencia_1_porciento'
]

CAPA_TERRENOS_R = 'Analitica_UT_Consolidada\R_TERRENO'
CAPA_TERRENOS_U = 'Analitica_UT_Consolidada\\U_TERRENO'

<h3>Generación de DF</h3>

In [ ]:
df_premarcas = pd.read_excel(RUTA_XLSX_PREMARCA, sheet_name='datos', dtype=TIPO_COLUMNA)

df_premarcas_inconsistencias_generales = df_premarcas[INCONSISTENCIAS_GENERALES]
df_premarcas_inconsistencias_geograficas = df_premarcas[INCONSISTENCIAS_GEOGRAFICAS]
df_premarcas_inconsistencias_salida_xlsx = df_premarcas[INCONSISTENCIAS_SALIDA_XLSX]

<h3>Exportación XLSX</h3>

In [ ]:
ruta_xlsx = os.path.join(RUTA_XLSX, NOMBRE_XLSX)

df_premarcas_inconsistencias_salida_xlsx.to_excel(ruta_xlsx, sheet_name='PreMarcas_MariaLaBaja')
print(f"Se exporta XLSX {NOMBRE_XLSX}")

<h3>Verificación de nulos</h3>

In [ ]:

conteo_npn_duplicados_marcas = df_premarcas_inconsistencias_generales['npn'].value_counts()

# ** Filtrar los valores que aparecen más de dos veces
resultado_repetidos_marcas = conteo_npn_duplicados_marcas[conteo_npn_duplicados_marcas > 2]

<h3>Espacialización Terrenos</h3>

In [ ]:

capa_terrenos_rurales = os.path.join(RUTA_FILEGDB, CAPA_TERRENOS_R)
capa_terrenos_urbanos = os.path.join(RUTA_FILEGDB, CAPA_TERRENOS_U)

# ** Terrenos Rurales
df_terreno_rural = pd.DataFrame.spatial.from_featureclass(capa_terrenos_rurales)

# ** Terrenos Urbanos
df_terreno_urbano = pd.DataFrame.spatial.from_featureclass(capa_terrenos_urbanos)

# ** Selección de columnas
columnas_selecciones = ['CODIGO',
                        'CODIGO_ANTERIOR',
                        'SHAPE']

# ** DF Terrenos Rurales, Urbanos
df_terreno_rural_reducido = df_terreno_rural[columnas_selecciones]
df_terreno_urbano_reducido = df_terreno_urbano[columnas_selecciones]

# ** Concatenación de dataframes
df_terrenos = pd.concat([df_terreno_rural_reducido, df_terreno_urbano_reducido])

<h3>No Mejoras\Informalidades</h3>

In [ ]:
df_marcas_formales = df_premarcas_inconsistencias_generales.query("condicion_predio == '0'")
df_marcas_formales_geograficas = df_premarcas_inconsistencias_geograficas.query("condicion_predio == '0'")

<h3>Mejoras\Informalidades</h3>

In [ ]:

df_marcas_informales = df_premarcas_inconsistencias_generales.query("condicion_predio == '5'")

df_marcas_informales = df_marcas_informales.copy()

In [ ]:

# ** Define una función que realice el slicing a un string, para eliminar el 1 al final de la cadena de caracteres
def eliminacion_mejora(text):
    # ? Función que me verifica si el tipo de dato parametrizado es el que ingresa a la función
    if isinstance(text, str) and text[21] == '5':
        cadena = text[:21] + '000000000'
        return cadena 
    else:
        return None

# ** Aplica la función a cada elemento de la columna npn y asigna el resultado a npn_parametrizado
df_marcas_informales['npn_parametrizado'] = df_marcas_informales['npn'].apply(eliminacion_mejora)

df_marcas_informales_conteo = df_marcas_informales['npn_parametrizado'].value_counts()

df_marcas_informales_conteo = df_marcas_informales_conteo[df_marcas_informales_conteo > 1]

df_marcas_informales_conteo = pd.DataFrame(df_marcas_informales_conteo).reset_index()

<h3>XSLX Informalidades</h3>

In [ ]:
df_marcas_informales_conteo = df_marcas_informales_conteo.rename(columns={'count': 'total_posibles_informalidades'})

ruta_informalidades_xlsx = os.path.join(RUTA_XLSX, NOMBRE_INFORMALIDADES_XLSX)

df_marcas_informales_conteo.to_excel(ruta_informalidades_xlsx, sheet_name='PreMarcas_Informalidades')
print(f"Se exporta XLSX {NOMBRE_INFORMALIDADES_XLSX}")

In [ ]:

# ** Terrenos - Formales
df_terrenos_marcas_formales = pd.merge(df_terrenos, df_marcas_formales, left_on='CODIGO', right_on='npn', how='inner')

# ** Terrenos - Diferencias Geométricas
df_terrenos_marcas_formales_geograficas = pd.merge(df_terrenos, df_marcas_formales_geograficas, left_on='CODIGO', right_on='npn', how='inner')

# ** Terrenos - Informales
df_terrenos_marcas_informales = pd.merge(df_terrenos, df_marcas_informales_conteo, left_on='CODIGO', right_on='npn_parametrizado', how='inner')

In [ ]:

conteo_join_w_espacializacion = df_terrenos_marcas_formales['CODIGO'].value_counts()

# ** Filtrar los valores que aparecen más de dos veces
resultado_join_w_geo = conteo_join_w_espacializacion[conteo_join_w_espacializacion > 2]

<h3>Preparación Analítica Formales Inconsistencias Alfanuméricas<h3>

In [ ]:

df_terrenos_marcas_formales['error_propietario'] = np.nan
df_terrenos_marcas_formales['error_documento'] = np.nan
df_terrenos_marcas_formales['error_direccion'] = np.nan
df_terrenos_marcas_formales['error_construccion'] = np.nan
df_terrenos_marcas_formales['error_destino'] = np.nan
df_terrenos_marcas_formales['error_inconsistencia_general'] = np.nan
df_terrenos_marcas_formales['error_inconsistencia_juridica'] = np.nan

df_terrenos_marcas_formales.loc[df_terrenos_marcas_formales['total_errores_de_propietario'] > 0, 'error_propietario'] = 'Errores en Propietarios'
df_terrenos_marcas_formales.loc[df_terrenos_marcas_formales['total_errores_de_documento'] > 0, 'error_documento'] = 'Errores en Documentos'
df_terrenos_marcas_formales.loc[df_terrenos_marcas_formales['total_errores_direccion'] > 0, 'error_direccion'] = 'Errores en la Dirección'
df_terrenos_marcas_formales.loc[df_terrenos_marcas_formales['total_errores_construcciones'] > 0, 'error_construccion'] = 'Errores en Construcciones'
df_terrenos_marcas_formales.loc[df_terrenos_marcas_formales['total_errores_destino'] > 0, 'error_destino'] = 'Error Destinacion Economica'
df_terrenos_marcas_formales.loc[df_terrenos_marcas_formales['total_inconsistencias_generales'] > 0, 'error_inconsistencia_general'] = 'Errores Generales'
df_terrenos_marcas_formales.loc[df_terrenos_marcas_formales['inconsistencias_juridicas'] > 0, 'error_inconsistencia_juridica'] = 'Con Inconsistencias Jurídicas'

<h3>Preparación Analítica Formales Inconsistencias Geográficas<h3>

In [ ]:
df_terrenos_marcas_formales_geograficas['diferencia_porcentaje'] = (df_terrenos_marcas_formales_geograficas['diferencia_porcentaje']*100).round(2)

df_terrenos_marcas_formales_geograficas = df_terrenos_marcas_formales_geograficas.query("error_diferencia_1_porciento == 1")

<h3>Estandarización de DF Marcas Formales</h3>

In [ ]:

columnas_a_concatenar = ['error_propietario', 
    'error_documento',
    'error_direccion',
    'error_construccion',
    'error_destino',
    'error_inconsistencia_general',
    'error_inconsistencia_juridica']

df_terrenos_marcas_formales['Analisis'] = df_terrenos_marcas_formales.apply(lambda x: ' - '.join([str(x[col]) for col in columnas_a_concatenar if pd.notnull(x[col]) and x[col] != '']), axis=1)

In [ ]:

# ** Exportación a BD
NOMBRE_CAPA_SALIDA = f"Analitica_UT_Consolidada\\{'premarcas_predios_formales'.upper()}"
ruta_salida_terrenos_marcas_formales = os.path.join(RUTA_FILEGDB, NOMBRE_CAPA_SALIDA)

df_terrenos_marcas_formales.spatial.to_featureclass(location=ruta_salida_terrenos_marcas_formales, overwrite=True)
print(f"Se exporta la capa {NOMBRE_CAPA_SALIDA}")

# ** Exportación a BD
NOMBRE_CAPA_SALIDA = f"Analitica_UT_Consolidada\\{'premarcas_predios_informales'.upper()}"
ruta_salida_terrenos_marcas_informales = os.path.join(RUTA_FILEGDB, NOMBRE_CAPA_SALIDA)

df_terrenos_marcas_informales.spatial.to_featureclass(location=ruta_salida_terrenos_marcas_informales, overwrite=True)
print(f"Se exporta la capa {NOMBRE_CAPA_SALIDA}")

# ** Exportación a BD
NOMBRE_CAPA_SALIDA = f"Analitica_UT_Consolidada\\{'premarcas_diferencia_areas'.upper()}"
ruta_salida_terrenos_diferencia_areas = os.path.join(RUTA_FILEGDB, NOMBRE_CAPA_SALIDA)

df_terrenos_marcas_formales_geograficas.spatial.to_featureclass(location=ruta_salida_terrenos_diferencia_areas, overwrite=True)
print(f"Se exporta la capa {NOMBRE_CAPA_SALIDA}")